In [1]:
from pymoo.termination.default import DefaultMultiObjectiveTermination, DefaultSingleObjectiveTermination
import warnings
import logging
from terminator import MyTermination
import itertools
from pymoo.optimize import minimize
from testSetup import Setup


logging.getLogger('matplotlib').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", ".*feasible.*")

In [2]:
# Test Configuration Lists
pop_list = [2000, 3000]  # population sizes
num_event_list = [20, 30, 50]  # trace lengths
declare_model_list = ["model1.decl", "model2.decl", "model3.decl", "model4.decl"]  # declare models
use_constraints = ["yes", "no"]
termination_map = {
    "my_termination": lambda pop_size: MyTermination(n_required=int(pop_size * 1.5)), # with the lambda it dynamically takes as a parameter the pop_size
    "multi": lambda _: DefaultMultiObjectiveTermination(
        xtol=1e-8, cvtol=1e-6, ftol=0.0025, period=30, n_max_gen=1000, n_max_evals=100000
    ),
    "single": lambda _: DefaultSingleObjectiveTermination(
        xtol=1e-8, cvtol=1e-6, ftol=1e-6, period=20, n_max_gen=1000, n_max_evals=100000
    ),
}


In [3]:
# TODO create a single file / add in the table all the numerics value of the fittness (diversity/n_events) / add the value of pymoo
# TODO add requirements file and Readme
# TODO save the final  population in a csv, 1 decoded trace per line, event;event ecc.
# TODO check if we can use default pymoo mut and crossover
# TODO to retrieve activity name, check d4py github


for test_run in range(1, 4):
    file_name = f"results/results2.csv"
    with open(file_name, "a") as f:
        f.write("ID,Population,TraceLength,Model,Termination,Algorithm, Constraints, "
                    f"ExecutionTime, DiversityScore, ConstraintScore, NumberViolationScore, Iteration{test_run}\n")
        ID = 1

        # iterate through all configurations
        for combination in itertools.product(pop_list, num_event_list, declare_model_list, termination_map.keys(), use_constraints):
            pop_size, trace_length, model, termination, constraints= combination

            print(f"Running with ID={ID}: Population={pop_size}, TraceLength={trace_length}, Model={model}, Termination={termination}, Constraints={constraints}")

            # exclude 'my_termination' if 'use_constraint' is "no"
            if constraints == "no" and termination == "my_termination":
                print(f"Skipping combination due to incompatible termination: {termination} with  {constraints} constraints")
                continue

            # initialize shared components and population
            (
                encoder, declare, event_log, dataframe, activities_name
            ) = Setup.initialize_shared_components(path_to_declareModel=f"../declare_models/{model}")
            (
                initial_population, initial_encoded_pop, features_range,
                lower_bounds, upper_bounds, mutation, crossover, sampling
            ) = Setup.setup_initial_population(trace_length, n_traces=10, activities_name=activities_name, encoder=encoder)

            # determine termination and algorithm configurations
            try:
                termination_instance = termination_map[termination](pop_size)
                pop_size_for_terminator = int(pop_size * 1.5) if termination == "my_termination" else pop_size
                algorithm_types = ["single", "multi"] if termination == "my_termination" else [termination]

                # logic to handle constraints
                if constraints == "no": # only run "single" GA algorithm if no constraints
                    algorithm_types = ["single"]
                elif constraints == "yes": # allow both "single" and "multi" GA
                    algorithm_types = ["single", "multi"]

                for algorithm_type in algorithm_types:
                    problem = Setup.create_problem(
                        algorithm_type, trace_length, encoder, declare, initial_encoded_pop,
                        lower_bounds, upper_bounds, event_log, dataframe, constraints
                    )

                    # init callback here because I need the obj data for plots
                    algorithm = Setup.create_algorithm(algorithm_type, problem, pop_size_for_terminator, sampling, crossover, mutation)

                    # run algorithm and logs
                    try:
                        result = minimize(problem, algorithm, termination=termination_instance, verbose=False)
                        exec_time = result.exec_time

                         # Extract data from the callback
                        data = result.algorithm.callback.get_data()
                        diversity_scores = data.get("diversity_history", None)
                        constraint_scores = data.get("constraint_history", None)
                        n_violations_scores = data.get("n_violations_history", None)
                        n_generations = data.get("generations", None)

                        # save log
                        Setup.log_results(f, ID, pop_size, trace_length, model, termination, algorithm_type, constraints,
                                          exec_time, diversity_scores, constraint_scores, n_violations_scores)
                        # plot and save progress
                        Setup.plot_and_save_progress(ID, test_run, algorithm_type, constraints,
                                                     diversity_scores, constraint_scores, n_violations_scores, n_generations)

                        Setup.save_feasible_traces(result.pop, encoder,test_run, ID, algorithm_type, constraints)

                    except Exception as algo_error:
                        Setup.log_results(f, ID, pop_size, trace_length, model, termination, algorithm_type, constraints,
                                          "N/A", diversity_scores, constraint_scores, n_violations_scores, error=algo_error)
                    ID += 1
            except Exception as config_error:
                Setup.log_results(f, ID, pop_size, trace_length, model, termination, algorithm_type, constraints,
                                          "N/A", diversity_scores, constraint_scores, n_violations_scores, error=algo_error)
                ID += 1


Running with ID=1: Population=2000, TraceLength=20, Model=model1.decl, Termination=my_termination, Constraints=yes
Execution Time (single): 53.47 seconds
Feasible traces saved to results/encoded_traces/ID_1_run_1_single_yes_constraints.csv
Execution Time (multi): 53.04 seconds
Feasible traces saved to results/encoded_traces/ID_2_run_1_multi_yes_constraints.csv
Running with ID=3: Population=2000, TraceLength=20, Model=model1.decl, Termination=my_termination, Constraints=no
Skipping combination due to incompatible termination: my_termination with  no constraints
Running with ID=3: Population=2000, TraceLength=20, Model=model1.decl, Termination=multi, Constraints=yes
Execution Time (single): 81.86 seconds
Feasible traces saved to results/encoded_traces/ID_3_run_1_single_yes_constraints.csv
Execution Time (multi): 85.39 seconds
Feasible traces saved to results/encoded_traces/ID_4_run_1_multi_yes_constraints.csv
Running with ID=5: Population=2000, TraceLength=20, Model=model1.decl, Terminat

KeyboardInterrupt: 